# bills_webscraper.ipynb

### Import python packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

import re, sqlite3
from collections import OrderedDict

import requests
from bs4 import BeautifulSoup
import selenium.webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

### Define Bill class

In [2]:
class Bill:
    
    def __init__(self, num, link, congress, short_title, long_title, date_filed, scope, status, author,
                 subject, pri_committee, sec_committee, date_lastUpdate, logs):
        self.num = num
        self.link = link
        self.congress = congress
        self.short_title = short_title
        self.long_title = long_title
        self.date_filed = date_filed
        self.scope = scope
        self.status = status
        self.author = author
        self.subject = subject
        self.pri_committee = pri_committee
        self.sec_committee = sec_committee
        self.date_lastUpdate = date_lastUpdate
        self.logs = logs
        self.entities = (num, link, congress, short_title, long_title, date_filed, scope, status, author,
                 subject, pri_committee, sec_committee, date_lastUpdate, logs)
        
    def __str__(self):
        return 'from ' + str(self.congress) + '_' + self.num + ': ' + self.short_title
    
    def insert_bill():
        pass
    
    def update_bill():
        pass
    
    def remove_bill():
        pass

### Compile Regex Patterns
These regex patterns will be used to extract relevant strings only from the webscraping function.

In [3]:
bill_number_regex = re.compile(r'(?<=\d\d&q=).*')
filed_date_regex = re.compile(r'(?<=Filed on ).*(?= by )')
author_regex = re.compile(r'(?<= by ).*(?=\n)')
status_regex = re.compile(r'.*(?= \()')
last_update_regex = re.compile(r'(?<=\().*(?=\))')
logs_regex = re.compile('(?<=\[ FIRST REGULAR SESSION, 18TH CONGRESS ]\n\[ \d\d\d\d ]\n).*(?=\(The legislative history)',
                       re.DOTALL)

### Define getBill function
getBill function is our web scraping script. It follows the following workflow: <br>
1. Access Home URL of 'senate.gov.ph' <br>
2. Collect all the bills' links on that page.<br>
3. Iterate over the gathered links.<br>
4. Click the "All Information" button and then scrape all the bills' informations from that frame.<br>
5. Go to next page from the Home URL and repeat all steps again.<br>

In [4]:
def getBill(congress_num, page_num):
    
    driver = selenium.webdriver.PhantomJS()
    
    home_url = 'http://www.senate.gov.ph/lis/leg_sys.aspx?congress='+str(congress_num)+'&type=bill&p='+str(page_num)
    res = requests.get(home_url)
    res.raise_for_status()
    print('Accessing '+ 'http://www.senate.gov.ph' + ' ...\n')
    
    soup = BeautifulSoup(res.content, 'html.parser')
    sublinks=[str(bill.attrs['href']) for bill in soup.select('a') if 'bill_res' in str(bill.attrs['href'])]
    print('There are ' + str(len(sublinks)) + ' bills on this page, '+ 'p' + str(page_num) + '.\n')
    
    bills_dict = OrderedDict()
    for sublink in sublinks:
        #
        link = 'http://www.senate.gov.ph/lis/'+sublink
        num = bill_number_regex.search(sublink).group()
        print("Trying to access child link "+ link + ' ...')
        driver.get(link)
        driver.implicitly_wait(100)
        
        allInfo_btn = driver.find_element_by_xpath("//a[@id='lbAll']")
        allInfo_btn.click()
        element = WebDriverWait(driver, 10).until(lambda x:x.find_element_by_xpath("//p[contains(text(),'Subject(s)')]")) 
        soup_allinfo = BeautifulSoup(driver.page_source, 'html.parser')
        
        short_title = soup_allinfo.select('p[class="h1_bold"]')[0].getText()
        allinfo_frame_text = soup_allinfo.select('td[id="content"]')[0].getText()
        date_filed = filed_date_regex.search(allinfo_frame_text).group()
        author = author_regex.search(allinfo_frame_text).group()
        long_title = soup_allinfo.select('blockquote')[0].getText()
        scope = soup_allinfo.select('blockquote')[1].getText()
        legis_status = soup_allinfo.select('blockquote')[2].getText()
        status = status_regex.search(legis_status).group()
        date_lastUpdate = last_update_regex.search(legis_status).group()
        subject = soup_allinfo.select('blockquote')[3].getText()
        pri_committee = soup_allinfo.select('blockquote')[4].getText()
        
        if len(soup_allinfo.select('blockquote'))==7:
            sec_committee = soup_allinfo.select('blockquote')[5].getText()
            logs = soup_allinfo.select('blockquote')[6].getText()
            logs = logs_regex.search(logs).group()
        else:
            sec_committee = ''
            logs = soup_allinfo.select('blockquote')[5].getText()
            logs = logs_regex.search(logs).group()
            
        bills_dict[num]=(num, link, congress_num, short_title, long_title, date_filed, scope, status, author,
                 subject, pri_committee, sec_committee, date_lastUpdate, logs)
        print('\t' + str(num) + ' has been successfully scraped.\n')
        
    print('All Bills have been successfully scraped on this page.')
    # Must close the ghostDriver before exiting this program...
    driver.quit()
    
    return bills_dict

### Define getMax_page function
This function will get the maximum/last page number of a Congressional Bill list.

In [5]:
def getMax_page(congress_num):
    driver = selenium.webdriver.PhantomJS()
    home_url = 'http://www.senate.gov.ph/lis/leg_sys.aspx?congress='+str(congress_num)+'&type=bill&p=1'
    clicks = 1
    driver.get(home_url)
    
    while True:
        try:
            more_btn = driver.find_element_by_xpath("//div[@id='pnl_NavBottom']//a[contains(text(),'Next')]")
            more_btn.click()
            bill_elem = WebDriverWait(driver, 10).until(lambda x:x.find_element_by_xpath("//span[@class='h1_sub']"))
            clicks += 1
            continue
        except NoSuchElementException:
            print('We reached last page... that is page ' + str(clicks) + '.')
            break
            
    driver.quit()
    return clicks

### Define Database functions
These database functions will create a database connection instance. It will create table within the database, check for records, insert records, as well as update existing records.

In [6]:
def connect_db(database):
    try:
        conn = sqlite3.connect(database)
        cursor = conn.cursor()
        return conn, cursor
    except sqlite3.Error as error:
        print("Error in connecting to sqlite3", error)

def create_bills_table(conn, cursor):
    cursor.execute("""CREATE TABLE if not exists senateBills(
                    num text PRIMARY KEY,
                    link text,
                    congress integer,
                    short_title text,
                    long_title text,
                    date_filed text,
                    scope text,
                    status integer,
                    author text,
                    subject text,
                    pri_committee text,
                    sec_committee text,
                    date_lastUpdate text,
                    logs text)""")
    
def check_bill_exists(bill_num, conn, cursor):
    with conn:
        cursor.execute("SELECT num FROM senateBills WHERE num = (?)",(bill_num,))
    rows = cursor.fetchall()
    return True if len(rows) else False

def insert_bill(entities, conn, cursor):
    with conn:
        cursor.execute("""INSERT INTO senateBills(
                        num, link, congress, short_title, long_title, date_filed, 
                        scope, status, author, subject, pri_committee, sec_committee, 
                        date_lastUpdate, logs) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",(entities))
        
def update_date_lastUpdate(bill_num, date_lastUpdate, conn, cursor):
    with conn:
        cursor.execute("UPDATE senateBills SET date_lastUpdate = (?) WHERE num = (?)",(date_lastUpdate, bill_num))

### Main Program

In [7]:
def scrape_thisCongress(congress):
    pages = list(range(1, getMax_page(congress) + 1, 1))
    conn, cursor = connect_db('phBills.db')
    create_bills_table(conn, cursor)
    bill_count = 0
    for page in pages:
        thisPage_bills = getBill(congress, page)
        for bill, contents in thisPage_bills.items():
            some_bill = Bill(*contents)
            if check_bill_exists(some_bill.num, conn, cursor):
                print(str(some_bill) + " already exists in our database. Updating 'date_lastUpdate' instead.")
                update_date_lastUpdate(some_bill.num, some_bill.date_lastUpdate, conn, cursor)
            else:
                insert_bill(some_bill.entities, conn, cursor)
            bill_count += 1
    conn.close()
    print('Collected ' + str(bill_count) + 'bills from ' + congress + 'th congress.')

In [27]:
congress_num = 18
page_num = 1
driver = selenium.webdriver.PhantomJS()
home_url = 'http://www.senate.gov.ph/lis/leg_sys.aspx?congress='+str(congress_num)+'&type=bill&p='+str(page_num)
res = requests.get(home_url)
res.raise_for_status()
print('Accessing '+ 'http://www.senate.gov.ph' + ' ...\n')

soup = BeautifulSoup(res.content, 'html.parser')
sublinks=[str(bill.attrs['href']) for bill in soup.select('a') if 'bill_res' in str(bill.attrs['href'])]
link = 'http://www.senate.gov.ph/lis/bill_res.aspx?congress=17&q=SBN-2234'

print("Trying to access child link "+ link + ' ...')
driver.get(link)
driver.implicitly_wait(100)
allInfo_btn = driver.find_element_by_xpath("//a[@id='lbAll']")
allInfo_btn.click()
element = WebDriverWait(driver, 10).until(lambda x:x.find_element_by_xpath("//p[contains(text(),'Long title')]"))   
soup_allinfo = BeautifulSoup(driver.page_source, 'html.parser')
# driver.quit()
# //p[contains(text(),'Secondary committee')]
print("\nDone!")

Accessing http://www.senate.gov.ph ...

Trying to access child link http://www.senate.gov.ph/lis/bill_res.aspx?congress=17&q=SBN-2234 ...

Done!


In [38]:
allcontents = [soup_allinfo.select('blockquote')[i].getText() for i in range(len(soup_allinfo.select('blockquote')))]

In [50]:
import pandas as pd
import sqlite3
con = sqlite3.connect("phBills.db")
df = pd.read_sql_query("SELECT * from senateBills", con)
con.close()

In [54]:
con = sqlite3.connect("phBills.db")

In [55]:
df = pd.read_sql_query("SELECT * from senateBills", con)

In [56]:
con.close()

In [65]:
print(df.iloc[2]['logs'])

5/27/2019
Prepared and submitted by the Committee on WAYS AND MEANS with Senator(s) EMMANUEL "MANNY" D. PACQUIAO, JOSEPH VICTOR G. EJERCITO, SHERWIN T. GATCHALIAN and JUAN EDGARDO "SONNY" M. ANGARA as author(s)  per Committee Report No. 714, recommending its approval in substitution of SBNos. 1599, 1605 and 2177, taking into consideration HBN-8677;
5/27/2019
Committee Report Calendared for Ordinary Business;
5/27/2019
Sponsor: Senator JUAN EDGARDO "SONNY" M. ANGARA;
5/27/2019
Transferred from the Calendar for Ordinary Business to the Calendar for Special Order;
5/27/2019
Sponsorship speech of Senator JUAN EDGARDO "SONNY" M. ANGARA;
5/27/2019
Co-sponsorship speech of Senators JOSEPH VICTOR G. EJERCITO, EMMANUEL "MANNY" D. PACQUIAO and SHERWIN T. GATCHALIAN;
5/27/2019
Remarks of Senator RISA HONTIVEROS;
5/27/2019
Interpellation of Senators VICENTE C. SOTTO III and RICHARD J. GORDON;
5/28/2019
Certified by the President of the Philippines, His Excellency President Rodrigo Roa Duterte, for

In [51]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("phBills.db")
df = pd.read_sql_query("SELECT * from senateBills", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

DatabaseError: Execution failed on sql 'SELECT * from senateBills': no such table: senateBills

In [47]:
print(x)

AN ACT AUTHORIZING THE SALE OF CERTAIN PARCELS OF LAND IN BARANGAY KRUS NA LIGAS, QUEZON CITY BY THE UNIVERSITY OF THE PHILIPPINES TO THE QUEZON CITY GOVERNMENT AMENDING FOR THE PURPOSE REPUBLIC ACT NO. 9500, OTHERWISE KNOWN AS THE UNIVERSITY OF THE PHILIPPINES CHARTER OF 2008 AND FOR OTHER PURPOSES
National
Approved by the President of the Philippines (6/30/2019)
University of the Philippines Charter (R.A.No.9500)
Education, Arts and Culture
Urban Planning, Housing and Resettlement          
No. 719 - Parcels of Land in Barangay Krus Na Ligas, Quezon City (5/27/2019)
Escudero, Francis "Chiz" G.
No Certification


Date

Parliamentary status

Senator(s)


5/28/2019
Sponsorship Speech
Escudero, Francis "Chiz" G.

5/28/2019
Interpellation
Drilon, Franklin M.

5/28/2019
Period of Interpellation Closed


5/28/2019
Period of Individual Amendments


5/28/2019
Period of Individual Amendments Closed


5/28/2019
Period of Committee Amendments Closed


5/28/2019
Period of Committee Amendments





In [21]:
soup_allinfo.select('blockquote')[-2].getText()

'\n\nDate\n\nParliamentary status\n\nSenator(s)\n\n\n11/5/2019\nSponsorship Speech\nPoe, Grace\n\n11/5/2019\nCo-Sponsorship Speech\nRecto, Ralph G.\n\n11/6/2019\nInterpellation\nDrilon, Franklin M.\n\n11/6/2019\nPeriod of Interpellation Closed\n\n\n11/6/2019\nPeriod of Committee Amendments\n\n\n11/6/2019\nPeriod of Committee Amendments Closed\n\n\n11/6/2019\nPeriod of Individual Amendments Closed\n\n\n'

In [22]:
'Entitled:' in soup_allinfo.select('blockquote')[-1].getText()

True

In [11]:
soup_allinfo.select('blockquote')[4].getText()

'\nEntitled:\nNATIONAL AUTISM CARE PLAN\n[ FIRST REGULAR SESSION, 18TH CONGRESS ]\n[ 2019 ]\n8/14/2019\nIntroduced by Senator RISA HONTIVEROS;\n8/15/2019\nWITHDRAWN.\n(The legislative history/plenary deliberations is prepared by the INDEXING, MONITORING AND LIS SECTION, LEGISLATIVE BILLS AND INDEX SERVICE)'

In [10]:
soup_allinfo.select('blockquote')[3].getText()

''

In [90]:
soup_allinfo.select('blockquote')[3].getText()

'Hospitals (Doh Retained)Hospitals, Increase Bed Capacity'

In [89]:
if 'Entitled:' in soup_allinfo.select('blockquote tbody')[1].getText():
    print(True)

IndexError: list index out of range

In [81]:
soup_allinfo.select('blockquote tbody')

[<tbody><tr><td><a href="journal.aspx?congress=16&amp;session=3R&amp;q=44">12/14/2015</a></td>
 <td>Period of Interpellation Closed</td>
 <td></td>
 </tr>
 <tr class="alt_color_g"><td><a href="journal.aspx?congress=16&amp;session=3R&amp;q=44">12/14/2015</a></td>
 <td>Period of Amendment Closed</td>
 <td></td>
 </tr>
 <tr><td><a href="journal.aspx?congress=16&amp;session=3R&amp;q=44">12/14/2015</a></td>
 <td>Sponsorship Speech</td>
 <td>Villar, Cynthia A.</td>
 </tr>
 </tbody>,
 <tbody><tr><td><blockquote>Angara, Juan Edgardo "Sonny" M.<br/>Aquino IV, Paolo Benigno "Bam"<br/>Binay, Maria Lourdes Nancy S.<br/>Cayetano, Alan Peter Companero S.<br/>Cayetano, Pia S.<br/>Drilon, Franklin M.<br/>Ejercito, Joseph Victor G.<br/>Enrile, Juan Ponce<br/>Escudero, Francis "Chiz" G.<br/>Guingona III, Teofisto "Tg"<br/>Honasan II, Gregorio B.<br/>Lapid, Manuel "Lito" M.<br/>Legarda, Loren B.<br/>Osmena III, Sergio R.<br/>Pimentel, Aquilino Koko III L.<br/>Poe, Grace L.<br/>Recto, Ralph G.<br/>Sotto I

In [38]:
x=soup_allinfo.select('p').index(soup_allinfo.find('p',text='Subject(s)'))
print(x)

40


In [41]:
len(soup_allinfo.select('p'))

51

In [42]:
len(soup_allinfo.select('blockquote'))

18

In [40]:
soup_allinfo.select('blockquote')[x]

IndexError: list index out of range

In [ ]:
soup_allinfo.select('p')

In [17]:
for x in soup_allinfo.select('p'):
    print(x.getText())

Rules of the Senate
History of the Senate
Symbols of Authority
Legislative Process
18th Congress Senators
List of Previous Senators
Roll of Senate President
Composition of the Senate
Terms of Office and Privileges
Committee Chairmanship 
Oversight/Ad hoc Committee Chairmanship 
Duties, Power & Jurisdiction
Committee Directory 
Oversight/Ad hoc Committee Directory 
Notice of Committee Meetings
Committee Schedule
Vision and Mission
Officers of the Secretariat
Office of the Secretary
Legislation
Administration and Financial Services
External Affairs and Relations
Office of the Sergeant-At-Arms
Bills
Resolutions
Journals
Committee Reports
Republic Acts
Adopted Resolutions
Treaties
Statistical Data
SEPO Publications
STSRO Publications
LBRMO Publications
ILS Publications
Other Publications
ECONOMIC AND FINANCIAL LITERACY ACT
Long title
Scope
Legislative status
Subject(s)
Primary committee
Committee report
Sponsor(s)
Document certification
Floor activity
Vote(s)
Legislative History
President'

In [15]:
soup_allinfo.select('p["Scope"]')

[]

In [ ]:
logs_wFootNote_regex = re.compile(r'(?<=\[ FIRST REGULAR SESSION, \d\d\w\w CONGRESS ]\n\[ \d\d\d\d ]\n).*(?=;)',re.DOTALL)

In [ ]:
def bills_logScraper(logs):
    try:
        out = logs_regex.search(logs).group()
    except AttributeError:
        out = logs_wFootNote_regex.search(logs).group()
    return out

In [ ]:
print(soup_allinfo.select('blockquote')[5].getText())

In [ ]:
bills_logScraper(soup_allinfo.select('blockquote')[-1].getText())

In [ ]:
len(soup_allinfo.select('blockquote'))

In [ ]:
soup_allinfo.select('blockquote')[-1]

In [ ]:
if len(soup_allinfo.select('blockquote'))==7:
    sec_committee = soup_allinfo.select('blockquote')[5].getText()
    logs = bills_logScraper(soup_allinfo.select('blockquote')[-1].getText())
else:
    sec_committee = ''
    logs = bills_logScraper(soup_allinfo.select('blockquote')[-1].getText())

In [ ]:
logs

In [ ]:
# Must close the ghostDriver before exiting this program...
driver.quit()